<a href="https://colab.research.google.com/github/alilotfi90/Flower_Classification/blob/main/flower_classification_test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter


!pip install efficientnet_pytorch
from zipfile import ZipFile
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import torchvision.models as models


drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile


with zipfile.ZipFile('/content/drive/MyDrive/flowerimages.zip', 'r') as zip_ref:
    zip_ref.extractall('unzipped_images_flower')

In [5]:
!pip install Pillow
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [7]:
from PIL import Image
import os

unzipped_folder = '/content/unzipped_images_flower/flowers'
output_folder = '/content/resized_images_flower'


os.makedirs(output_folder, exist_ok=True)

subdirectories = [d for d in os.listdir(unzipped_folder) if os.path.isdir(os.path.join(unzipped_folder, d))]

seen = set()
for flower_category in subdirectories:
    input_folder = os.path.join(unzipped_folder, flower_category)
    output_category_folder = os.path.join(output_folder, flower_category)


    os.makedirs(output_category_folder, exist_ok=True)

    image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

    for image_file in image_files:
        input_image_path = os.path.join(input_folder, image_file)
        output_image_path = os.path.join(output_category_folder, image_file)

        image = Image.open(input_image_path)

        # Resize the image
        target_size = (456, 456)
        image = image.resize(target_size, Image.ANTIALIAS)

        # Convert the image to grayscale
        image = image.convert('L')


        image.save(output_image_path)

        width, height = image.size
        seen.add((width, height))

<ipython-input-7-4b12688be33c>:30: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(target_size, Image.ANTIALIAS)


In [8]:
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image


from sklearn.model_selection import train_test_split


max_images_per_class =40

data = []

for flower_category in subdirectories:
    category_folder = os.path.join(output_folder, flower_category)
    image_files = [f for f in os.listdir(category_folder) if f.endswith('.jpg')]
    label = subdirectories.index(flower_category)  # Assign a label based on the folder name

    # Limit the number of images to max_images_per_class
    image_files = image_files[:max_images_per_class]

    for image_file in image_files:
        image_path = os.path.join(category_folder, image_file)
        data.append((image_path, label))

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


# Adjusting transform to work with grayscale images
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel grayscale
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        return image, label


train_dataset = CustomDataset(train_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)





In [12]:
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=16)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)



max_epoch = [10, 20, 30]
for e in max_epoch:
    model.train()
    # Training
    for epoch in range(e):
        print(epoch)
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 10 == 9:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 50))
                running_loss = 0.0


    # Testing
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print("for number of epochs being:", e)
    print('Accuracy of the network on the test images: %d %%' % (
        100 * correct / total))

    # Save the trained model along with other relevant information
    torch.save({
        'epoch': e,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, f"/content/drive/MyDrive/images/flower_efficient{e}")


Loaded pretrained weights for efficientnet-b0
0
[1,    10] loss: 0.554
1
[2,    10] loss: 0.542
2
[3,    10] loss: 0.517
3
[4,    10] loss: 0.500
4
[5,    10] loss: 0.473
5
[6,    10] loss: 0.451
6
[7,    10] loss: 0.429
7
[8,    10] loss: 0.409
8
[9,    10] loss: 0.383
9
[10,    10] loss: 0.370
for number of epochs being: 10
Accuracy of the network on the test images: 52 %
0
[1,    10] loss: 0.346
1
[2,    10] loss: 0.327
2
[3,    10] loss: 0.299
3
[4,    10] loss: 0.286
4
[5,    10] loss: 0.273
5
[6,    10] loss: 0.256
6
[7,    10] loss: 0.237
7
[8,    10] loss: 0.218
8
[9,    10] loss: 0.216
9
[10,    10] loss: 0.204
10
[11,    10] loss: 0.177
11
[12,    10] loss: 0.175
12
[13,    10] loss: 0.167
13
[14,    10] loss: 0.152
14
[15,    10] loss: 0.164
15
[16,    10] loss: 0.144
16
[17,    10] loss: 0.140
17
[18,    10] loss: 0.130
18
[19,    10] loss: 0.120
19
[20,    10] loss: 0.110
for number of epochs being: 20
Accuracy of the network on the test images: 77 %
0
[1,    10] loss: 0.1